In [72]:
import os
import sqlite3
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import dateutil
import pylab as py
import seaborn as sns
import scipy 
from scipy import stats
import sklearn.metrics
from numpy  import array
import glob
import functools
from functools import reduce
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid.inset_locator import (inset_axes, InsetPosition, mark_inset)
import statsmodels.api as sm
import matplotlib.patheffects as path_effects
import matplotlib.lines as mlines
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [261]:
data0=pd.read_csv(r'C:\Users\uqjojeda\Nextcloud\PURTERRA-A0131\2020\USDA\extracted3.csv')

In [262]:
data0.drop(['asd_desc', 'domain_desc', 'begin_code', 'freq_desc',
       'statisticcat_desc', 'watershed_desc', 'commodity_desc',
       'util_practice_desc', 'state_ansi', 'country_code',
       'reference_period_desc', 'zip_5', 'source_desc', 'class_desc',
       'watershed_code', 'sector_desc', 'congr_district_code', 'county_ansi',
       'location_desc', 'group_desc', 'state_name', 'asd_code',
       'agg_level_desc', 'week_ending', 'end_code', 'load_time', 'region_desc', 'CV (%)',
       'domaincat_desc', 'unit_desc', 'prodn_practice_desc'], axis=1, inplace=True)

new_names_dict = {'state_alpha': 'state',
                  'state_fips_code': 'stateID',
                  'Value': 'grain_yield',
                  'county_name': 'county',
                  'county_code': 'countyID'}
data0.rename(index=str, columns=new_names_dict, inplace=True)

#data['yield_tn']=(data['yield_bu']/14.8697).round(2)
# convert county names to uppercase
#data['county'] = data['county'].str.lower()
#data['county'] = data['county'].str.capitalize() 

In [213]:
data0.short_desc.unique()

array(['SORGHUM - ACRES PLANTED', 'SORGHUM, IRRIGATED - ACRES PLANTED',
       'SORGHUM, NON-IRRIGATED - ACRES PLANTED'], dtype=object)

In [263]:
data=data0.loc[data0['short_desc'] == "SORGHUM - ACRES PLANTED"]

In [230]:
data.dtypes

year             int64
grain_yield      int64
stateID          int64
county          object
countyID         int64
state           object
country_name    object
short_desc      object
dtype: object

In [264]:
data['st_co'] = data['stateID'].astype(str) + "_" + data['countyID'].astype(str)
data['area_ha'] = data['grain_yield']/0.404686/1000000
#data['grain_yield_tn'] = data['grain_yield']*25.4/0.404686/1000
#data['grain_yield_tn'] = data['grain_yield']/0.404686
#data.grain_yield_tn = data.grain_yield_tn.round(1)

<ipython-input-264-673d52106b40>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['st_co'] = data['stateID'].astype(str) + "_" + data['countyID'].astype(str)
<ipython-input-264-673d52106b40>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['area_ha'] = data['grain_yield']/0.404686/1000000


In [265]:
data

,year,grain_yield,stateID,county,countyID,state,country_name,short_desc,st_co,area_ha
0,1940,112000,40,BEAVER,7,OK,UNITED STATES,SORGHUM - ACRES PLANTED,40_7,0.276758
1,1940,141500,40,CIMARRON,25,OK,UNITED STATES,SORGHUM - ACRES PLANTED,40_25,0.349654
2,1940,110900,40,ELLIS,45,OK,UNITED STATES,SORGHUM - ACRES PLANTED,40_45,0.274040
3,1940,60600,40,HARPER,59,OK,UNITED STATES,SORGHUM - ACRES PLANTED,40_59,0.149746
4,1940,121000,40,TEXAS,139,OK,UNITED STATES,SORGHUM - ACRES PLANTED,40_139,0.298997
...,...,...,...,...,...,...,...,...,...,...
79124,2019,35000,48,JIM WELLS,249,TX,UNITED STATES,SORGHUM - ACRES PLANTED,48_249,0.086487
79125,2019,18000,48,OTHER (COMBINED) COUNTIES,998,TX,UNITED STATES,SORGHUM - ACRES PLANTED,48_998,0.044479
79126,2019,54200,48,CAMERON,61,TX,UNITED STATES,SORGHUM - ACRES PLANTED,48_61,0.133931
79127,2019,75500,48,WILLACY,489,TX,UNITED STATES,SORGHUM - ACRES PLANTED,48_489,0.186564


In [105]:
data.short_desc.unique()

array(['SORGHUM, GRAIN - YIELD, MEASURED IN BU / ACRE',
       'SORGHUM, SILAGE - YIELD, MEASURED IN TONS / ACRE',
       'SORGHUM, GRAIN, IRRIGATED - YIELD, MEASURED IN BU / ACRE',
       'SORGHUM, GRAIN, NON-IRRIGATED - YIELD, MEASURED IN BU / ACRE',
       'SORGHUM, SILAGE, IRRIGATED - YIELD, MEASURED IN TONS / ACRE',
       'SORGHUM, SILAGE, NON-IRRIGATED - YIELD, MEASURED IN TONS / ACRE',
       'SORGHUM, GRAIN - YIELD, MEASURED IN BU / NET PLANTED ACRE',
       'SORGHUM, GRAIN, IRRIGATED - YIELD, MEASURED IN BU / NET PLANTED ACRE',
       'SORGHUM, GRAIN, NON-IRRIGATED - YIELD, MEASURED IN BU / NET PLANTED ACRE'],
      dtype=object)

In [ ]:
#data.year.unique()
#data.short_desc.unique()
#data2 = data.loc[(data['year'] >= 2009) & (data['year'] <= 2019)]
#data2.year.unique()

In [266]:
grain2 = data.groupby('st_co', as_index=False).sum()
grain2.drop(['year'], axis=1, inplace=True)
grain2.round(2)

,st_co,grain_yield,stateID,countyID,area_ha
0,10_1,2800,50,5,0.01
1,10_3,100,10,3,0.00
2,10_5,8100,50,25,0.02
3,13_1,32260,234,18,0.08
4,13_101,540,39,303,0.00
...,...,...,...,...,...
2058,8_91,110,24,273,0.00
2059,8_93,600,24,279,0.00
2060,8_95,339280,360,4275,0.84
2061,8_99,2727670,424,5247,6.74


In [254]:
grain2.describe()

,grain_yield,stateID,countyID,area_ha
count,2.063000e+03,2063.000000,2063.00000,2063.000000
mean,3.761176e+05,799.035870,4687.28793,0.044595
std,9.924546e+05,975.055269,19756.63879,0.771050
min,1.000000e+01,1.000000,1.00000,0.000000
25%,3.110000e+03,186.000000,538.00000,0.000000
50%,2.122000e+04,459.000000,1599.00000,0.000000
75%,1.879000e+05,1260.000000,4085.00000,0.000000
max,1.225210e+07,22464.000000,467064.00000,26.000000


In [267]:
grain2.to_csv(r'C:\Users\uqjojeda\Nextcloud\PURTERRA-A0131\2020\USDA\PlantedArea.csv', index=None, mode='a')

In [143]:
grain2

,st_co,grain_yield,stateID,countyID,grain_yield_tn
0,17_119,60.500000,17.0,119.0,3.800000
1,17_121,58.600000,17.0,121.0,3.700000
2,17_127,94.000000,17.0,127.0,5.900000
3,17_133,78.900000,17.0,133.0,4.933333
4,17_157,93.100000,17.0,157.0,5.833333
...,...,...,...,...,...
442,8_89,64.000000,8.0,89.0,4.000000
443,8_9,46.050000,8.0,9.0,2.900000
444,8_95,55.400000,8.0,95.0,3.500000
445,8_99,42.633333,8.0,99.0,2.666667
